# Intent Recognition

In this practical, we will learn how to apply the HuggingFace Transformers library to our own Intent Recognition task for our chatbot.

####**NOTE: Be sure to set your runtime to a GPU instance!**

## Install the Hugging Face Transformers Library

Run the following cell below to install the transformers library.

In [1]:
# !pip install transformers==4.42.4
!pip install transformers

In [2]:
!pip install tf-keras==2.17.0
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'
import tensorflow as tf
print(tf.__version__)

2.17.0


## Getting the data and prepare the data

In [3]:
import pandas as pd

data_url = 'https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/datasets/airchat_intents.csv'
df = pd.read_csv(data_url)

df.head()

,Label,Text
0,atis_abbreviation,what is fare code h
1,atis_abbreviation,what is booking class c
2,atis_abbreviation,what does fare code q mean
3,atis_abbreviation,what is fare code qw
4,atis_abbreviation,what does the fare code f mean


We noticed that there are two columns 'Label' and 'Text'. Let's just examine what are the different labels we have and how many samples we have for each labels.

In [4]:
df['Label'].value_counts()

,count
Label,
atis_flight,3666
atis_airfare,423
atis_ground_service,255
atis_airline,157
atis_abbreviation,147
atis_yes,82
atis_aircraft,81
atis_no,67
atis_flight_time,54


We can see that some labels have very few sample such as 'atis_meal', 'atis_airline#atis_flight_no', 'atis_cheapest', and so on. With so few samples, our model will have difficulty in learning any meaningful pattern from it. We will group these labels (with few samples) into a new label called 'others'.  

---



### Re-define our Classification Labels

Here we define the labels we are interested in classifying based on the original labels, and also we added a new label called 'Others'.


In [5]:
# Create a list of unique labels that we will recognize.
#
sentence_labels = [
              "others",
              "atis_abbreviation",
              "atis_aircraft",
              "atis_airfare",
              "atis_airline",
              "atis_flight",
              "atis_flight_time",
              "atis_greeting",
              "atis_ground_service",
              "atis_quantity",
              "atis_yes",
              "atis_no"]

# This creates a reverse mapping dictionary of "label" -> index.
#
sentence_labels_id_by_label = dict((t, i) for i, t in enumerate(sentence_labels))

Now we will map the previous labels to the few ones we specified in the cell above. We will also convert the text labels into numeric labels (e.g. others->0, atis_abbreviation->1, etc). We can use the `map()` function in dataframe to help us do that. We define a lambda function that do the mapping.

In [6]:
df['Label'] = df['Label'].map(lambda label:
                              sentence_labels_id_by_label[label]
                              if label in sentence_labels_id_by_label
                              else 0)

In [7]:
# examine a few random samples
df.sample(10)

,Label,Text
4413,5,i need a flight from kansas city to newark on...
2237,5,show me all canadian airlines flights from to...
2327,5,show me flights on september twenty sixth please
4116,5,give me the latest flight tomorrow on america...
1283,5,okay tell me what the earliest flight is that...
4929,8,show me ground transport in seattle
2109,5,flights from newark new jersey to cleveland ohio
2413,5,what are the flights from pittsburgh to denve...
4375,5,a listing of all flights from boston to balti...
2333,5,okay how about a flight on sunday from tampa ...


### Split Our Data

We will now separate the texts and labels and call them all_texts and all_labels and we will split the dataset into training and validation set. We do a stratified split to ensure we have equal representation of different labels in both train and validation set.

In [8]:
all_texts = df['Text']
all_labels = df['Label']

In [9]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(all_texts,
                                                                    all_labels,
                                                                    test_size=0.2,
                                                                    stratify=all_labels)

In [10]:
train_labels.value_counts()/len(train_labels)

,count
Label,
5,0.707770
3,0.081564
8,0.049228
4,0.030405
1,0.028475
0,0.027751
10,0.015927
2,0.015685
11,0.012790


### Tokenize the text

Before we can use the text for classification, we need to tokenize them. We will use Tokenizer of the pretrained model 'distilbert-base-uncased' as we will be fine-tunining on a pretrained model 'distilbert-base-uncased'.


In [11]:
len(sentence_labels)

12

In [12]:
## before we can feed the texts to tokenizer, we need to convert our texts into list of text string instead of
## panda Series. We can do this by using to_list().

train_texts = train_texts.to_list()
train_labels = train_labels.to_list()
val_texts = val_texts.to_list()
val_labels = val_labels.to_list()

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
train_encodings = tokenizer(train_texts, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, padding=True, truncation=True)

In [15]:
train_encodings[0]


Encoding(num_tokens=52, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

Once we have the encodings, we will go ahead and create a tensorflow dataset, ready to be used to train our model. Since the HuggingFace pretrained model (the tensorflow version) is a Keras model, it can consume the tf.data dataset.

In [16]:
import tensorflow as tf

batch_size = 16

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(batch_size)

## Train Your Sentence Classification Model

Run the following cell to download the "distilbert-base-uncased" and perform fine-tuning training using the dataset that we have above.

In [17]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=len(sentence_labels))

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

As in previous lab, we start with a smaller learning rate 5e-5 (0.00005) and slowly reduce the learning rate over the course of training.

In [18]:

from tensorflow.keras.optimizers.schedules import PolynomialDecay

num_epochs = 2

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Since our dataset is already batched, we can simply take the len.
num_train_steps = len(train_dataset) * num_epochs

lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

In [19]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy


opt = Adam(learning_rate=lr_scheduler)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

model.fit(train_dataset, validation_data=val_dataset, epochs=num_epochs)

Epoch 1/2
259/259 [==============================] - 51s 109ms/step - loss: 0.5029 - accuracy: 0.8772 - val_loss: 0.1610 - val_accuracy: 0.9575
Epoch 2/2
259/259 [==============================] - 25s 96ms/step - loss: 0.1218 - accuracy: 0.9768 - val_loss: 0.1014 - val_accuracy: 0.9826


### Evaluating the Model

Run the following code to evaluate our model with entire validation data set.

We also print out the classification report to see how the model performs for each label. Note that those with smaller number of samples typically have lower F1-score.


In [20]:
output = model.predict(val_dataset, batch_size=1)
pred_probs = tf.nn.softmax(output.logits, axis=-1)
preds = tf.argmax(pred_probs, axis=-1)

65/65 [==============================] - 4s 27ms/step


In [21]:
val_labels = []
for _, labels in val_dataset.as_numpy_iterator():
    val_labels.extend(labels)

In [22]:
from sklearn.metrics import classification_report

print(classification_report(val_labels, preds))

              precision    recall  f1-score   support

           0       0.95      0.72      0.82        29
           1       0.96      0.93      0.95        29
           2       1.00      0.94      0.97        16
           3       0.97      1.00      0.98        85
           4       0.91      0.97      0.94        31
           5       0.99      1.00      1.00       733
           6       1.00      1.00      1.00        11
           7       1.00      0.91      0.95        11
           8       0.98      0.94      0.96        51
           9       0.83      1.00      0.91        10
          10       0.89      1.00      0.94        16
          11       1.00      0.93      0.96        14

    accuracy                           0.98      1036
   macro avg       0.96      0.94      0.95      1036
weighted avg       0.98      0.98      0.98      1036



### Saving the Model

When you training has completed, run the following cell to save your model.

Remember to download the model from Google Colab if you want to use later.

In [23]:
# Save the model

model.save_pretrained("intent_model")

## Putting Our Model to the Test

Run the following cell to create the necessary classes and functions to load our model and perform inference.


In [24]:
# Import the necessary libraries
#
from transformers import (
    AutoTokenizer,
    TFAutoModelForSequenceClassification
)

# Create the DistilBERT tokenizer
#
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Define a function to perform inference on a single input text.
#
def infer_intent(model, text):
    # Passes the text into the tokenizer
    #
    input = tokenizer(text, truncation=True, padding=True, return_tensors="tf")

    # Sends the result from the tokenizer into our classification model
    #
    output = model(input)

    # Extract the output logits and convert to softmax
    # Find the classification index with the highest value.
    #
    pred_label = tf.argmax(tf.nn.softmax(output.logits, axis=-1), axis=-1)

    return pred_label

# Create a list of unique labels that we will recognize.
# Obviously this has to match what we trained our model with
# earlier.
#
sentence_labels = [
              "others",
              "atis_abbreviation",
              "atis_aircraft",
              "atis_airfare",
              "atis_airline",
              "atis_flight",
              "atis_flight_time",
              "atis_greeting",
              "atis_ground_service",
              "atis_quantity",
              "atis_yes",
              "atis_no"]

# Load the saved model file
#
intent_model = TFAutoModelForSequenceClassification.from_pretrained("intent_model")



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some layers from the model checkpoint at intent_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

In [25]:
text = input()

print (sentence_labels[infer_intent(intent_model, text)[0]])

I want go new york
atis_flight
